## Problem 3

In [1]:
! pip install yfinance -q

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('constituents.csv')
data
print("Shape of data: ", data.shape)
symbol_list = data.Symbol.tolist()

Shape of data:  (505, 3)


In [4]:
import pandas_datareader.data as web
import yfinance as yf
yf.pdr_override()

start_date = '2012-06-11'
end_date = '2022-06-09'
df = web.get_data_yahoo(symbol_list, start=start_date, end = end_date)

[                       1%                       ]  3 of 505 completedGot error from yahoo api for ticker VIAC, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
[************          26%                       ]  132 of 505 completedGot error from yahoo api for ticker CERN, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
[********************  41%                       ]  205 of 505 completedGot error from yahoo api for ticker XLNX, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
[********************* 43%                       ]  218 of 505 completedGot error from yahoo api for ticker BRK.B, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
[**********************46%                       ]  231 of 505 completedGot error from yahoo api for ticker DISCK, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
[****

In [5]:
adj_close_data = df['Adj Close']
print("Shape of Data: ", adj_close_data.shape)

Shape of Data:  (2516, 505)


In [6]:
limit = 0.05
drop_list = []
for column in adj_close_data.columns:
  if adj_close_data[column].isnull().mean() > limit:
    drop_list.append(column)

adj_close_data.drop(drop_list, axis=1, inplace=True)
print(adj_close_data.shape)

(2516, 453)


In [7]:
adj_close_data.dropna(axis=0, inplace=True)
print(adj_close_data.shape)

(2429, 453)


In [8]:
adj_close_data.index = pd.to_datetime(adj_close_data.index)

In [9]:
# 2014년 1월 1일 이후의 모든 날에 대해 12-1 모멘텀을 계산
momentum_df = adj_close_data.copy()

for col in momentum_df.columns:
  momentum_df[col] = adj_close_data[col].pct_change(252) - adj_close_data[col].pct_change(21)
momentum_df = momentum_df[momentum_df.index >= '2014-01-01']

In [10]:
# 12-1 모멘텀을 계산하고, 상위 20% 주식은 long을 하고, 하위 20% 주식은 short을 하기로함
q_df = momentum_df.copy()
for i in range(momentum_df.shape[0]):
  q_df.iloc[i,:] = pd.qcut(momentum_df.iloc[i,:], 5, labels=[1,2,3,4,5])

signal_df = q_df.copy()
signal_df[:] = 0
signal_df[q_df == 1] = -1
signal_df[q_df == 5] = 1

In [11]:
signal_df

,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,0.0,1.0,0.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,-1.0,1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-01-03,0.0,1.0,0.0,-1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,-1.0,1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-01-06,0.0,1.0,0.0,-1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,-1.0,1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-01-07,0.0,1.0,0.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,-1.0,1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-01-08,0.0,1.0,0.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,-1.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-02,0.0,-1.0,0.0,1.0,1.0,0.0,0.0,0.0,-1.0,0.0,...,1.0,-1.0,0.0,1.0,-1.0,-1.0,0.0,-1.0,-1.0,0.0
2022-06-03,0.0,-1.0,0.0,1.0,1.0,0.0,0.0,0.0,-1.0,0.0,...,1.0,-1.0,0.0,1.0,-1.0,-1.0,0.0,-1.0,-1.0,0.0
2022-06-06,0.0,-1.0,0.0,1.0,1.0,0.0,0.0,0.0,-1.0,0.0,...,1.0,-1.0,0.0,1.0,-1.0,-1.0,0.0,-1.0,-1.0,0.0


In [12]:
# Equally weighted Portfolio를 사용하기로함(이 때, 전체 주식 수의 20%를 사용하기로 함)
weight_df = signal_df * (1/(signal_df.shape[1]//5))

In [13]:
# Use equally weighted strategy
weight_df

,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,0.0,0.011111,0.0,-0.011111,0.011111,0.011111,0.0,0.0,0.000000,0.0,...,-0.011111,0.011111,-0.011111,-0.011111,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2014-01-03,0.0,0.011111,0.0,-0.011111,0.011111,0.011111,0.0,0.0,0.011111,0.0,...,-0.011111,0.011111,-0.011111,-0.011111,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2014-01-06,0.0,0.011111,0.0,-0.011111,0.011111,0.011111,0.0,0.0,0.011111,0.0,...,-0.011111,0.011111,-0.011111,-0.011111,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2014-01-07,0.0,0.011111,0.0,-0.011111,0.011111,0.011111,0.0,0.0,0.000000,0.0,...,-0.011111,0.011111,-0.011111,-0.011111,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2014-01-08,0.0,0.011111,0.0,-0.011111,0.011111,0.011111,0.0,0.0,0.000000,0.0,...,-0.011111,0.011111,-0.011111,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-02,0.0,-0.011111,0.0,0.011111,0.011111,0.000000,0.0,0.0,-0.011111,0.0,...,0.011111,-0.011111,0.000000,0.011111,-0.011111,-0.011111,0.0,-0.011111,-0.011111,0.0
2022-06-03,0.0,-0.011111,0.0,0.011111,0.011111,0.000000,0.0,0.0,-0.011111,0.0,...,0.011111,-0.011111,0.000000,0.011111,-0.011111,-0.011111,0.0,-0.011111,-0.011111,0.0
2022-06-06,0.0,-0.011111,0.0,0.011111,0.011111,0.000000,0.0,0.0,-0.011111,0.0,...,0.011111,-0.011111,0.000000,0.011111,-0.011111,-0.011111,0.0,-0.011111,-0.011111,0.0


In [14]:
# 이제 rebalancing을 구현해야한다
# rebalancing은 메월(22일) 주기로 진행하기로 한다.
monthly_rebalanced_df = weight_df.iloc[0:len(weight_df):21,:]
monthly_rebalanced_df.head()

,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,0.0,0.011111,0.000000,-0.011111,0.011111,0.011111,0.0,0.000000,0.000000,0.0,...,-0.011111,0.011111,-0.011111,-0.011111,0.0,0.0,0.000000,0.0,0.000000,0.0
2014-02-03,0.0,0.011111,0.011111,0.000000,0.011111,0.011111,0.0,0.000000,0.011111,0.0,...,-0.011111,0.011111,-0.011111,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0
2014-03-05,0.0,0.011111,0.011111,0.000000,0.000000,0.011111,0.0,0.000000,0.011111,0.0,...,-0.011111,0.011111,-0.011111,0.000000,0.0,0.0,-0.011111,0.0,0.000000,0.0
2014-04-03,0.0,0.011111,0.011111,0.000000,0.000000,0.011111,0.0,0.000000,0.011111,0.0,...,-0.011111,0.011111,-0.011111,0.000000,0.0,0.0,0.000000,0.0,0.011111,0.0
2014-05-05,0.0,0.011111,0.011111,0.000000,0.000000,0.000000,0.0,-0.011111,0.000000,0.0,...,-0.011111,0.011111,-0.011111,0.000000,0.0,0.0,0.000000,0.0,0.011111,0.0


In [15]:
# 매달 rebalancing을 한 것을 누적한 결과
monthly_rebalanced_df.cumsum()

,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,0.000000,0.011111,0.000000,-0.011111,0.011111,0.011111,0.000000,0.000000,0.000000,0.000000,...,-0.011111,0.011111,-0.011111,-0.011111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2014-02-03,0.000000,0.022222,0.011111,-0.011111,0.022222,0.022222,0.000000,0.000000,0.011111,0.000000,...,-0.022222,0.022222,-0.022222,-0.011111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2014-03-05,0.000000,0.033333,0.022222,-0.011111,0.022222,0.033333,0.000000,0.000000,0.022222,0.000000,...,-0.033333,0.033333,-0.033333,-0.011111,0.000000,0.000000,-0.011111,0.000000,0.000000,0.000000
2014-04-03,0.000000,0.044444,0.033333,-0.011111,0.022222,0.044444,0.000000,0.000000,0.033333,0.000000,...,-0.044444,0.044444,-0.044444,-0.011111,0.000000,0.000000,-0.011111,0.000000,0.011111,0.000000
2014-05-05,0.000000,0.055556,0.044444,-0.011111,0.022222,0.044444,0.000000,-0.011111,0.033333,0.000000,...,-0.055556,0.055556,-0.055556,-0.011111,0.000000,0.000000,-0.011111,0.000000,0.022222,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-03,0.133333,-0.211111,0.044444,0.411111,-0.055556,0.355556,0.144444,0.088889,0.544444,0.044444,...,-0.222222,-0.155556,0.022222,-0.388889,-0.155556,0.122222,0.022222,-0.155556,0.544444,-0.133333
2022-03-07,0.133333,-0.222222,0.044444,0.422222,-0.055556,0.355556,0.144444,0.100000,0.544444,0.044444,...,-0.222222,-0.166667,0.022222,-0.377778,-0.166667,0.111111,0.022222,-0.166667,0.544444,-0.133333
2022-04-05,0.133333,-0.233333,0.044444,0.433333,-0.044444,0.355556,0.144444,0.100000,0.544444,0.044444,...,-0.222222,-0.177778,0.022222,-0.366667,-0.177778,0.100000,0.022222,-0.177778,0.533333,-0.133333
